In [1]:
import pickle
import networkx as nx 
import numpy as np
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

In [2]:
with open('../data/MAG_CS/embeddings_llama7b_wnet_pretrain_last.pickle', 'rb') as f:
    embeddings = pickle.load(f)

with open('../data/MAG_CS/test_nodes.pickle', 'rb') as f:
    test_nodes = pickle.load(f)

G_gold = nx.read_edgelist('../data/MAG_CS/all.edgelist', create_using=nx.DiGraph, delimiter='\t')

In [3]:
len(embeddings)

29484

In [4]:
word_names = list(embeddings.keys())
values = np.vstack(list(embeddings.values()))

In [5]:
len(word_names), len(test_nodes)

(29484, 1000)

In [8]:
total_found = 0
mrr = 0
mr = 0
for i in tqdm(range(100)):
    child, parents = test_nodes[i]

    child_embedding = embeddings[child]
    nn = NearestNeighbors(n_neighbors=1000, metric='euclidean', algorithm='brute').fit(values)
    dists, idxs = nn.kneighbors(child_embedding[np.newaxis,:])

    predicted_names = []
    for idx in idxs[0]:
        predicted_names.append(word_names[idx])

    found = 0
    for gt in parents:
        found += (gt in predicted_names)

    if found == 0:
       # print(child, ': not found')
        mrr += 0

    else:
        best_pos = 1000
        for gt in parents:
            if (gt in predicted_names):
                pos = predicted_names.index(gt)
                if pos < best_pos:
                    best_pos = pos
     #   print(child, ': found', '; best pos = ', best_pos)

        mrr += 1/ (best_pos + 1)
        mr += (best_pos + 1)
        total_found += 1

print(total_found, mrr, mrr / 1000, mr / 1000)

 29%|██▉       | 29/100 [00:05<00:13,  5.27it/s]


KeyboardInterrupt: 

In [21]:
i = 0
child, parents = test_nodes[i]

child_embedding = embeddings[child]
nn = NearestNeighbors(n_neighbors=5000, metric='euclidean', algorithm='brute').fit(values)
dists, idxs = nn.kneighbors(child_embedding[np.newaxis,:])

predicted_names = []
for idx in idxs[0]:
    predicted_names.append(word_names[idx])

In [22]:
predicted_names[:5]

['spanish verbs',
 'french verbs',
 'german verbs',
 'english verbs',
 'arabic verbs']

In [25]:
if parents[0] in predicted_names:
    print(predicted_names.index(parents[0]))

277


True


In [5]:
def nearest_neighbors(values, all_values, nbr_neighbors=10):
    nn = NearestNeighbors(nbr_neighbors, metric='cosine', algorithm='brute').fit(all_values)
    dists, idxs = nn.kneighbors(values)

AtlasView({'bispherical coordinates': {}, 'horizontal coordinate system': {}, 'bipolar cylindrical coordinates': {}, 'conical coordinates': {}, 'toroidal coordinates': {}, 'skew coordinates': {}, 'paraboloidal coordinates': {}, 'coordinate rotations and reflections': {}, 'elliptic cylindrical coordinates': {}})

In [6]:
test_gt = []

for node in test_nodes:
    parents = list(G_gold.predecessors(node))
    test_gt.append((node, parents))

In [7]:
test_gt[1]

('effective transmission rate', ['channel', 'wireless'])

In [ ]:
with open('../data/MAG_CS/test_nodes.pickle', 'wb') as f:
    